In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#read in transaction dataset
df3 = pd.read_csv('C:\Documents\Rifornimenti_Carburante_ClubQ8.csv', sep = ';', decimal=",")


df3[['REQUEST_YEAR','REQUEST_TIME']] = df3.DATA_OPERAZIONE.str.split(" ",expand=True,)


df3 = df3.drop(columns="DATA_OPERAZIONE")

df3.columns = ['CUSTOMER_CODE', 'STATION_CODE', 'PRODUCT', 'SALES_MODALITY', 'LITRES', 'LOYALTY_POINTS_AWARDED','REQUEST_YEAR','REQUEST_TIME']

df3

,CUSTOMER_CODE,STATION_CODE,PRODUCT,SALES_MODALITY,LITRES,LOYALTY_POINTS_AWARDED,REQUEST_YEAR,REQUEST_TIME
0,7071161603600606750,584,CARICO GASOLIO,EASY,7,3,2021-03-01,00:00:57
1,7071161603000039982,155,CARICO GASOLIO,SELF,17,17,2021-03-01,00:02:53
2,7071161603000720656,4803,CARICO GASOLIO,OPT,39,19,2021-03-01,00:03:10
3,7071161603014646483,2491,CARICO HIPERFORM DIESEL,SERVITO,54,54,2021-03-01,00:05:45
4,7071161603011692357,8265,CARICO GPL,SERVITO,31,31,2021-03-01,00:07:29
...,...,...,...,...,...,...,...,...
11787857,7071161603670650324,1114,CARICO GASOLIO,SELF,50,50,2021-08-31,23:58:22
11787858,7071161603020697769,7064,CARICO BENZINA,EASY,12,6,2021-08-31,23:58:26
11787859,7071161603010864247,7933,CARICO BENZINA,OPT,9,4,2021-08-31,23:58:56
11787860,7071161603601882848,5693,CARICO GASOLIO,SERVITO,50,50,2021-08-31,23:59:03


In [3]:
#define function to count number and percentage of Na values
def summarize_na(df: pd.DataFrame) -> pd.DataFrame:
    nan_count = df.isna().sum()
    return pd.DataFrame({'nan_count': nan_count, 
                         'nan_pct': nan_count / len(df) * 100
                         }
                        )[nan_count > 0]

In [4]:
df3_nan_sum = summarize_na(df3)
df3_nan_sum

,nan_count,nan_pct


In [31]:
def get_frequencies(series: pd.Series, n_categories: int = None, 
                    bins: int = None, dropna: bool = True
                    ) -> pd.DataFrame:
    """Return a DataFrame displaying the series frequencies.
    
    Parameters
    ----------
    series: pd.Series
        The series for which frequencies are to be computed.
    n_categories: int, optional
        Maximum number of categories to return in output.
        Low-frequency categories will be grouped together to
        reach this threshold.
        
    """
    vc = series.value_counts(ascending=False, 
                             bins=bins, 
                             dropna=dropna
                             )
    if n_categories is not None:
        if not isinstance(n_categories, int) or n_categories <= 0:
            raise TypeError('n_categories should be a strictly positive integer')
        if n_categories < len(vc):
            freq_others = vc.iloc[n_categories - 1:].sum()
            vc = vc.iloc[:n_categories - 1]\
                   .append(pd.Series({'others': freq_others}))
    return pd.DataFrame({'absolute': vc, 
                         'relative': vc / len(series),
                         },
                        index=vc.index
                        )

In [32]:
#proportion for each station code
get_frequencies(df3['STATION_CODE'], n_categories=5)

,absolute,relative
7132,76192,0.006464
4522,61540,0.005221
68,58743,0.004983
138,51564,0.004374
others,11539823,0.978958


In [33]:
#proportion for each product 
get_frequencies(df['PRODUCT'], n_categories=5)

,absolute,relative
CARICO GASOLIO,5835363,0.495031
CARICO BENZINA,4403314,0.373546
CARICO GPL,641881,0.054453
CARICO HIPERFORM DIESEL,551675,0.046800
others,355629,0.030169


In [34]:
#proportion for mode of sales
get_frequencies(df['SALES_MODALITY'], n_categories=5)

,absolute,relative
SELF,6167973,0.523248
SERVITO,4706534,0.399270
EASY,546173,0.046334
OPT,348523,0.029566
OPT POS,18659,0.001583


In [8]:
#descriptive statistics for the number of litres being bought
df3['LITRES'].describe().apply("{0:.5f}".format)

count    11787862.00000
mean           40.55607
std            45.62399
min             1.00000
25%            20.00000
50%            33.00000
75%            50.00000
max          1200.00000
Name: LITRES, dtype: object

In [9]:
#descriptive statistics of the variable loyalty points
df3['LOYALTY_POINTS_AWARDED'].describe().apply("{0:.5f}".format)

count    11787862.00000
mean           37.23561
std            25.89148
min          -180.00000
25%            18.00000
50%            31.00000
75%            50.00000
max           240.00000
Name: LOYALTY_POINTS_AWARDED, dtype: object

In [10]:
#read in customer database
df2 = pd.read_csv('C:\Documents\Anagrafica_ClubQ8.csv', sep = ';', decimal=",")

df2['DATE_JOINED'] = df2['DATA_BATTESIMO']

df2 = df2.drop(columns="DATA_BATTESIMO")

df2.columns = ['CUSTOMER_CODE', 'SEX', 'DOB', 'REGION', 'PROVINCE', 'CITY', 'TYPE_OF_CARD' ,'TOTAL_POINTS', 'DATE_JOINED']

df2

,CUSTOMER_CODE,SEX,DOB,REGION,PROVINCE,CITY,TYPE_OF_CARD,TOTAL_POINTS,DATE_JOINED
0,7071161603018922195,M,1951-09-14 00:00:00,LAZIO,VITERBO,SUTRI,STARQ8_FISICA,511.0,2015-01-08 12:32:58
1,7071161603005992565,F,1988-09-15 00:00:00,NaN,NaN,NaN,STARQ8_FISICA,379.0,2020-07-10 12:22:35
2,7071161603025867243,F,1943-09-09 00:00:00,LAZIO,ROMA,ROMA,STARQ8_FISICA,108.0,2020-07-10 18:07:14
3,7071161603021997861,M,1974-03-01 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,100.0,2020-07-10 18:08:21
4,7071161603025554049,M,1963-11-22 00:00:00,CAMPANIA,SALERNO,EBOLI,STARQ8_FISICA,284.0,2020-07-10 18:09:27
...,...,...,...,...,...,...,...,...,...
1656132,7071161603672630225,F,1962-06-22 00:00:00,EMILIA ROMAGNA,RIMINI,SAN GIOVANNI IN MARIGNANO,STARQ8_FISICA,2741.0,2019-11-27 18:06:31
1656133,7071161603673015798,M,1967-03-19 00:00:00,LAZIO,ROMA,ROMA,STARQ8_FISICA,8996.0,2019-03-21 08:34:30
1656134,7071161603024035909,M,1987-03-18 00:00:00,SICILIA,PALERMO,PALERMO,STARQ8_FISICA,424.0,2021-01-19 09:07:37
1656135,7071161603601002132,F,1984-09-20 00:00:00,LOMBARDIA,BRESCIA,ISEO,STARQ8_VIRTUALE,740.0,2020-12-15 22:57:21


In [22]:
#merge transaction and customer database
df = pd.merge(df3, df2, how = 'left')

In [12]:
df

,CUSTOMER_CODE,STATION_CODE,PRODUCT,SALES_MODALITY,LITRES,LOYALTY_POINTS_AWARDED,REQUEST_YEAR,REQUEST_TIME,SEX,DOB,REGION,PROVINCE,CITY,TYPE_OF_CARD,TOTAL_POINTS,DATE_JOINED
0,7071161603600606750,584,CARICO GASOLIO,EASY,7,3,2021-03-01,00:00:57,M,1957-05-13 00:00:00,PIEMONTE,TORINO,TORINO,STARQ8_VIRTUALE,457.0,2020-06-01 15:12:55
1,7071161603000039982,155,CARICO GASOLIO,SELF,17,17,2021-03-01,00:02:53,M,1961-02-20 00:00:00,PIEMONTE,TORINO,CIRIE',STARQ8_FISICA,1198.0,2021-02-01 10:50:35
2,7071161603000720656,4803,CARICO GASOLIO,OPT,39,19,2021-03-01,00:03:10,M,1992-08-11 00:00:00,LIGURIA,LA SPEZIA,SARZANA,STARQ8_FISICA,1837.0,2020-07-09 08:18:23
3,7071161603014646483,2491,CARICO HIPERFORM DIESEL,SERVITO,54,54,2021-03-01,00:05:45,F,1979-04-25 00:00:00,VENETO,PADOVA,PADOVA,STARQ8_FISICA,1803.0,2015-03-11 22:28:35
4,7071161603011692357,8265,CARICO GPL,SERVITO,31,31,2021-03-01,00:07:29,M,1971-03-27 00:00:00,LAZIO,ROMA,ROMA,STARQ8_FISICA,17568.0,2001-10-21 17:43:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11787857,7071161603670650324,1114,CARICO GASOLIO,SELF,50,50,2021-08-31,23:58:22,M,1965-10-15 00:00:00,SICILIA,ENNA,PIAZZA ARMERINA,STARQ8_FISICA,22415.0,2013-09-08 07:59:48
11787858,7071161603020697769,7064,CARICO BENZINA,EASY,12,6,2021-08-31,23:58:26,M,1993-09-03 00:00:00,CAMPANIA,NAPOLI,NAPOLI,STARQ8_FISICA,283.0,2021-06-22 17:35:49
11787859,7071161603010864247,7933,CARICO BENZINA,OPT,9,4,2021-08-31,23:58:56,M,1977-03-17 00:00:00,CAMPANIA,SALERNO,EBOLI,STARQ8_FISICA,328.0,2021-08-18 11:12:32
11787860,7071161603601882848,5693,CARICO GASOLIO,SERVITO,50,50,2021-08-31,23:59:03,M,1997-04-11 00:00:00,LAZIO,ROMA,ROMA,STARQ8_VIRTUALE,2430.0,2021-08-31 23:06:09


In [13]:
#look at differeces in total points across males and females
t = df.groupby('SEX')[['TOTAL_POINTS']].agg('mean')
t

,TOTAL_POINTS
SEX,
F,2382.701396
M,3337.432468


This part is for future analysis for now, we will add purchase frequencies for customers and conduct an RFM analysis

In [15]:
df['REQUEST_YEAR'] = pd.to_datetime(df['REQUEST_YEAR'])

In [29]:
df['sum', 'count'] = df.groupby('CUSTOMER_CODE')['LITRES'].agg(['sum', 'count'])
df

ValueError: Wrong number of items passed 2, placement implies 1

In [24]:
customerdata = pd.merge(df, frequency, how = 'left')
customerdata

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [20]:
# Aggregate at customer level
customer_data = pd.DataFrame(df.pivot_table(index="CUSTOMER_CODE", 
                                              values=["REQUEST_YEAR", "LITRES"], 
                                              aggfunc={"REQUEST_YEAR": [min, max, pd.Series.nunique], "LITRES": sum}))
customer_data.head()

KeyboardInterrupt: 